## SHA1

In [1]:
import numpy as np

In [11]:
def stringToBinary(s):
    res = list(''.join(format(ord(i),'08b') for i in s))
    return res

In [12]:
stringToBinary('Ge')

['0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1']

In [6]:
def stringToBinaryP(M):
    M =list( ''.join(format(ord(i), '08b') for i in M))
    return M

In [7]:
stringToBinaryP('Ge')

['0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1']

In [ ]:
def leftshift(a,b):
    return ((a << b) | (a >> (32 - b))) & 0xffffffff

In [124]:
def stringToBinary(s):
    res = list(''.join(format(ord(i),'08b') for i in s))
    return res
def padding(M):
    if len(M)<512:
        print("Padding!")
        length = list(bin(len(M)))[2:]
        M.append('1')
        for i in range(len(M),512-len(length)):
            M.append('0')
        M.extend(length)
    return M
def leftRotate(a,b):
    return ((a << b) | (a >> (32 - b))) & 0xffffffff
def f1(B,C,D):
    t=D^(B&(C^D))
    return t
def f2(B,C,D):
    t=B^C^D
    return t
def f3(B,C,D):
    t=(B&C)|(B&D)|(C&D)
    return t
def f4(B,C,D):
    t=B^C^D
    return t
def helperFunctionSHA1 (M,F):
    a = F[0]
    b = F[1]
    c = F[2]
    d = F[3]
    e = F[4]
    a1 = a
    b1 = b
    c1 = c
    d1 = d
    e1 = e
    M = padding(M)
    Words = [] # this will contain the words
    for i in range(16):
        t = M[32*i:32*(i+1)]
        t = int(''.join(map(str,t)),2)# this will convert to binary 
        Words.append(t)
    for i in range(16,80):
        t=Words[i-16]^Words[i-14]
        t=t^Words[i-8]
        t=t^Words[i-3]
        t = leftRotate(t,1)
        Words.append(t)
    for i in range(80):
        if i>=0 and i<20:
            f = f1(b,c,d)
            k = 0x5A827999
        elif i>=20 and i<40:
            f = f2(b,c,d)
            k = 0x6ED9EBA1
        elif i>= 40 and i<60:
            f = f3(b,c,d)
            k = 0x8F1BBCDC
        else:
            f = f4(b,c,d)
            k = 0xCA62C1D6
        temp = (leftshift(a,5)+f+e+Words[i]+k)&0xffffffff
        e = d
        d = c
        c = leftRotate(b,30)
        b = a
        a = temp
    a = (a+a1)& 0xffffffff
    b = (b+b1)& 0xffffffff
    c = (c+c1)& 0xffffffff
    d = (d+d1)& 0xffffffff
    e = (e+e1)& 0xffffffff
    return [a,b,c,d,e]

In [125]:
def sha1M(M):
    Message_in_binary = stringToBinary(M)
    A = 0x67452301
    B = 0xEFCDAB89
    C = 0x98BADCFE
    D = 0x10325476
    E = 0xC3D2E1F0
    F = [A,B,C,D,E]
    for i in range(0,len(Message_in_binary),256):
        a = min(i+256,len(Message_in_binary))
        F = helperFunctionSHA1(Message_in_binary[i:a],F)
    print('%08x%08x%08x%08x%08x'%(F[0],F[1],F[2],F[3],F[4]))
    return F

In [126]:
sha1M('welcome')

Padding!
c0b137fe2d792459f26ff763cce44574a5b5ab03


[3232839678, 762913881, 4067424099, 3437512052, 2780146435]

## MITM

In [116]:
key = "hello"
message = "welcome"
F1 = sha1M(key)
messageBin = stringToBinary(message)
F2 = helperFunctionSHA1(messageBin,F1)
print(message,'%08x%08x%08x%08x%08x'%(F2[0],F2[1],F2[2],F2[3],F2[4]))
r = "rand"
rand = stringToBinary(r)
F3 = helperFunctionSHA1(rand,F2)
print(message+r,'%08x%08x%08x%08x%08x'%(F3[0],F3[1],F3[2],F3[3],F3[4]))
message1 = "welcomerand"
message1Bin = stringToBinary(message1)
F4 = helperFunctionSHA1(message1Bin,F1)
print(message+r,'%08x%08x%08x%08x%08x'%(F4[0],F4[1],F4[2],F4[3],F4[4]))

aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
welcome 29e81ecd91380deaaf46a58227613dc5d0a115a0
welcomerand fe5dfac277b1c0dc1c14a3a316ade1214db7e200
welcomerand 89ca8a9e3ace8132bf5b93d2aff7263d1d89b9b1


## HMAC

In [131]:
q=['0','0','1','1','0','1','1','0']
o=['0','1','0','1','1','1','0','0']
ipad=[]
opad=[]
for i in range(64):
    ipad.extend(q)
    opad.extend(o)
def XOR (a,b):
    res = []
    for i in range(len(a)):
        if a[i] == b[i]:
            res.append('0')
        else:
            res.append('1')
    return res
key = "hello"
message = "Welcome"
keyBin = stringToBinary(key)
for i in range(len(keyBin),512):
    keyBin.append('0')
Si = XOR(keyBin,ipad)
So = XOR(keyBin,opad)
messageBin = stringToBinary(message)
A = 0x67452301
B = 0xEFCDAB89
C = 0x98BADCFE
D = 0x10325476
E = 0xC3D2E1F0
F = [A,B,C,D,E]
F1 = helperFunctionSHA1(keyBin,F)
F2 = helperFunctionSHA1(messageBin,F1)
print('%08x%08x%08x%08x%08x'%(F2[0],F2[1],F2[2],F2[3],F2[4]))
def intTobin(a):
    c=[]
    for j in a:
        c.extend([i for i in list('{0:8b}'.format(j))])
    return c
M2=intTobin(F2)
F1=helperFunctionSHA1(So,F)
F2=helperFunctionSHA1(M2,F1)
print('%08x%08x%08x%08x%08x'%(F2[0],F2[1],F2[2],F2[3],F2[4]))

Padding!
1cdc16327cb96fdbdc635c280f7587c5264ea6f2
Padding!
e3c45c4bec33704b8db52ca82034834935772597


In [129]:
F2

[484185650, 2092527579, 3697499176, 259360709, 642688754]

In [119]:
fruits

['apple', 'banana', 'cherry', 'Ford', 'BMW', 'Volvo']

## RSA

In [132]:
import math
n = a = 160
b = 3
t1 = 0
t2 = 1
q = 0
r = 0
t = 0
g = 0
if math.gcd(a,b) != 1:
    print("There is no multiplicative inverse of the given a nd b as their gcd not 1 or they are not co primes")
    print("gcd(a,b) =",math.gcd(a,b))
    g = 1
print("q \t a \t b \t r \t t1 \t t2 \t t")
while b!= 0:
    q = math.floor(a/b)
    r = a - (b*q)
    t = t1 - (q*t2)
    print(q,"\t",a,"\t",b,"\t",r,"\t",t1,"\t",t2,"\t",t)
    a = b
    b = r
    t1 = t2
    t2 = t
q = "-"
t = "-"
r = "-"
print(q,"\t",a,"\t",b,"\t",r,"\t",t1,"\t",t2,"\t",t)
print("The value at t1 is the inverse ",t1)
if t1<0:
    t1 = t1 +n
    print("The positive value of inverse is",t1)
if g == 1:
    print("Clearly t1 * b % a not 1",(t1*b)%a)

q 	 a 	 b 	 r 	 t1 	 t2 	 t
53 	 160 	 3 	 1 	 0 	 1 	 -53
3 	 3 	 1 	 0 	 1 	 -53 	 160
- 	 1 	 0 	 - 	 -53 	 160 	 -
The value at t1 is the inverse  -53
The positive value of inverse is 107


In [140]:
import math
def eucledianExtended(a,b):
    if math.gcd(a,b)!=1:
        return -1
    n = a 
    t1 = 0
    t2 = 1
    while b!=0:
        q = math.floor(a/b)
        r = a - b*q
        t = t1 - t2 * q
        t1 = t2
        t2 = t
        a = b
        b = r
    if t1<0:
        t1 = t1+n
    return t1
eucledianExtended(3,8)

2

In [145]:
def modularInverse (a,b):
    for i in range(2,b):
        if ((a%b)*(i%b))%b == 1:
            return i
p = 7
q = 3
n = p*q
phin = (p-1)*(q-1)
e  = 0
for i in range(2,phin):
    if math.gcd(phin,i) == 1:
        e = i
        break
d = modularInverse(e,phin)
print("Public Key ",e,n)
print("Private Key",d,n)
message = 17
cipher = (message ** e)%n
decipher = (cipher ** d)%n
print(cipher,decipher,message)

Public Key  5 21
Private Key 5 21
5 17 17


## Digital Signature using RSA

In [147]:
def modularInverse(a,b):
    for i in range(2,b):
        if ((a%b)*(i%b))%b == 1:
            return i
p = 7
q = 3
n = p*q
phi = (p-1)*(q-1)
e = 0
for i in range(2,phi):
    if math.gcd(phi,i) == 1:
        e = i
        break
d = modularInverse(e,phi)
message = 12
cipher = (message ** e)%n
decipher = (cipher ** d)%n
signature = (message ** d)%n
print("Alice sent Digital Signature {",message,"|",signature,"}")
message_ret = (signature ** e)%n
if message_ret == message:
    print("Verified")

Alice sent Digital Signature { 12 | 3 }
Verified


## ElGamal

In [149]:
# Bob choses a Zp where p is a prime
# g is a generator from Zp
# a is secret key from Zp
# h = g^a
# Public key (p,g,Zp,h)
# Alice chooses y from Zp
# s = h^y 
# c1 = g^y
# c2 = Message * s
# Bob decrypts by 
# c2/c1^a
import math
import random
p = 17
Zp = [i for i in range(1,p)]
g = 3 # generator for Zp
a = random.choice(Zp)
print("Private key of Bob is",a)
h = g**a
print("Public Key of Bob is (",p,g,h,Zp,")")
y = random.choice(Zp)# Alice chooses a random number from the group
c1 = g**y
message = 5
s = h**y
c2 = message * s
print("Message sent by Alice (",c1,c2,")")
s_d = c1**a
message_dec = c2/s_d
print("Message decrypted by Bob",message_dec)


Private key of Bob is 15
Public Key of Bob is ( 17 3 14348907 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] )
Message sent by Alice ( 531441 380886740229331961696448638603077808752124007011975983620007828724785686715165190098005 )
Message decrypted by Bob 5.0


## Elgamal Digital Signatures

In [156]:
# Bob will choose a private key from Zp-1 {1,... p-2}
# y = g**x mod p
# k is again choose random from {2,...,p-2} and k is relatively prime to p-1 ! very very important
# r = g**k mod p
# s = (H(m) - xr) k^(-1) mod (p-1)! very  very very important
# (r,s) is the signature from Bob
# Verification
# g^H(m) = y**r r**s mod p
def modInverse(a,b):
    for i in range(1,b):
        if ((a%b)*(i%b))%b == 1:
            return i
p = 17
g = 3
Zp = [i for i in range(1,p)]
a = random.choice(Zp)# Secret key of Bob
y = (g**a)%p
message = 5
Zp2 = [i for i in range(2,p-1)] 
k = random.choice(Zp2)
while math.gcd(p-1,k) != 1:
    k = random.choice(Zp2) 
r = (g**k)%p
s = ((message - a*r)%(p-1)*modInverse(k,(p-1))%(p-1))%(p-1)
print("bob sends Digital Signature (r,s),(",r,s,")")
print("Public keys of Bob (",y,p,g,Zp,")")
decrypt = ((y**r)%p*(r**s)%p)%p
if decrypt == (g**message)%p:
    print("Verified")
print(decrypt)

bob sends Digital Signature (r,s),( 12 13 )
Public keys of Bob ( 12 17 3 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] )
Verified
5


## Dffie Hellman MITM

In [157]:
# P,G are known to all
# P --> prime number
# G --> generator
# a --> private key of Alice
# b --> private key of Bob
# Pb = G**b mod P
# Pa = G**a mod P
p = 17
g = 3
a = 5 # Alice private key
b = 3 # Bob private key
public_key_bob = (g**b)%p
public_key_alice = (g**a)%p
shared = (public_key_bob ** a)%p
print("Public Key Bob (",public_key_bob,")")
print("Public Key Alice (",public_key_alice,")")
print("Shared private key (",shared,")")

Public Key Bob ( 10 )
Public Key Alice ( 5 )
Shared private key ( 6 )


## Birthday Paradox

In [162]:
t = val = 1
while True:
    val = val * (1-(t/365))
    if 1-val >= 0.5:
        break
    t += 1
print(t)

22


In [163]:
364/365

0.9972602739726028

## ECC

In [3]:
import math
import random
import numpy as np
def modInverse(a,b):
    for i in range(1,b):
        if(((a%b)*(i%b))%b == 1):
            return i
    return -1
def pointDoubling(xp,yp,a,b,p):
    s = (3*(xp**2)+a)%p*modInverse(2*yp,p)
    xr = (s**2 - 2*xp)%p
    yr = (s*(xp-xr)-yp)%p
    return (xr,yr)
def pointAddition(xp,xq,yp,yq,p):
    s = (yp-yq)%p*modInverse(xp-xq,p)
    xr = (s**2 - (xp+xq))%p
    yr = (s*(xp-xr)-yp)%p
    return [xr,yr]
def pointAddition1(xp,xq,yp,yq,p):
    s = (yp-yq)%p*modInverse(xp-xq,p)
    xr = (s**2 - (xp+xq))
    yr = (s*(xp-xr)-yp)%p
    return [xr,yr]
def pointCalculator(xp,yp,k):
    arr = []
    arr.append(0)
    arr.append([xp,yp])
    for i in range(2,k+1):
        if i%2 == 1:
            xr,yr = pointAddition(arr[i-1][0],xp,arr[i-1][1],yp,p)
            arr.append([xr,yr])
        else:
            xr,yr = pointDoubling(arr[int(i/2)][0],arr[int(i/2)][1],a,b,p)
            arr.append([xr,yr])
    return [arr[k][0],arr[k][1]]
def satisfy(x,p,a,b):
    val = x**3 + a*(x) +b
    val = math.sqrt((val%p))
    temp = math.floor(val)
    if temp == val:
        return temp
    return -1
def mtoPm(m,K,p,a,b):
    j = 0
    while satisfy(m*K+j,p,a,b) == -1:
        j += 1
    Pm = satisfy(m*K+j,p,a,b)
    return [m*K+j,Pm]
a = 1
b = 1
p = 4177
xp = 0
yp = 1
K = 30
secret_key_A = 5
secret_key_B = 4
Pa = pointCalculator(xp,yp,secret_key_A)# Shared public key of A
Pb = pointCalculator(xp,yp,secret_key_B)# Shared public key of B
print("a which is coefficient of x in Equation of curve",a)
print("b Constant in Equation of curve",b)
print("p The modular limit",p)
print("P or B the base Point","[",xp,",",yp,"]")
print("K the common value which is available to publicly",K)
print("Pa public key of A",Pa)
print("Pb public key of B",Pb)
print("The things that will be available publicly are (",a,",",b,",",p,",","[",xp,",",yp,"]",",",K,",",Pa,",",Pb,",",")")
# A will choose some random k with which he will send the value
message = 2174
Pmpoint = mtoPm(message,K,p,a,b)
print("Pm calculated by A is ",Pmpoint)
k = random.randint(2,p-1)
kB = pointCalculator(xp,yp,k)
kPb = pointCalculator(Pb[0],Pb[1],k)
encrypted = pointAddition(Pmpoint[0],kPb[0],Pmpoint[1],kPb[1],p)
print("The message that will be sent is [kB,(Pm+kPb)] [",kB,encrypted,"]")
pkB = pointCalculator(kB[0],kB[1],secret_key_B)
pkB[1] = -pkB[1] + p # As in subtraction we need to inverse sign of the y co ordinates
decrypted = pointAddition(encrypted[0],pkB[0],encrypted[1],pkB[1],p)
print("This is the decrypted point",decrypted)

a which is coefficient of x in Equation of curve 1
b Constant in Equation of curve 1
p The modular limit 4177
P or B the base Point [ 0 , 1 ]
K the common value which is available to publicly 30
Pa public key of A [2866, 2128]
Pb public key of B [1853, 1124]
The things that will be available publicly are ( 1 , 1 , 4177 , [ 0 , 1 ] , 30 , [2866, 2128] , [1853, 1124] , )
Pm calculated by A is  [65320, 7]
The message that will be sent is [kB,(Pm+kPb)] [ [1783, 180] [694, 342] ]
This is the decrypted point [2665, 7]


284